In [1]:
# import all dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Float, Integer
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
from config import password

In [2]:
# create engine for connection to local host
protocol = 'postgresql'
username = 'postgres'
password = password
host = 'localhost'
port = 5432
database_name = 'Project-2'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [3]:
# read first csv
travel_csv = "Resources/Travel_by_month.csv"
travel_df =pd.read_csv(travel_csv)
travel_df.head(30)

,Date,Number of Trips 1-3 (Sum),Number of Trips 3-5 (Sum),Number of Trips 5-10 (Sum),Number of Trips 10-25 (Sum),Number of Trips 25-50 (Sum),Number of Trips 50-100 (Sum)
0,Jan-19,"26,834,466,455","12,952,315,594","15,950,096,166","15,199,029,960","5,076,183,145","1,676,952,407"
1,Feb-19,"24,347,566,662","11,870,026,967","14,719,257,863","14,093,668,108","4,715,357,077","1,572,076,828"
2,Mar-19,"30,942,105,172","14,991,418,422","18,737,410,376","18,267,432,391","5,967,630,335","1,902,728,755"
3,Apr-19,"33,425,468,112","16,067,286,787","20,196,735,426","20,041,766,804","6,405,800,428","1,924,130,243"
4,May-19,"34,318,175,900","16,488,446,849","20,778,297,665","20,303,221,871","6,116,174,341","1,740,648,434"
5,Jun-19,"33,144,615,919","15,826,367,634","20,107,819,242","19,982,403,330","6,120,145,714","1,736,008,631"
6,Jul-19,"34,073,651,703","16,608,557,358","21,445,182,698","21,400,512,787","6,470,530,864","1,780,450,737"
7,Aug-19,"34,682,611,907","17,029,842,787","21,904,879,064","21,703,307,957","6,504,577,940","1,776,071,341"
8,Sep-19,"33,672,680,754","16,612,759,623","21,312,780,831","21,186,318,598","6,308,923,522","1,690,127,364"
9,Oct-19,"31,071,467,186","15,748,293,509","20,445,722,197","20,789,971,887","6,074,019,393","1,522,017,494"


In [4]:
# check columns
travel_df.columns

Index(['Date', 'Number of Trips 1-3 (Sum)', 'Number of Trips 3-5 (Sum)',
       'Number of Trips 5-10 (Sum)', 'Number of Trips 10-25 (Sum)',
       'Number of Trips 25-50 (Sum)', 'Number of Trips 50-100 (Sum)'],
      dtype='object')

In [5]:
# rename columns for export to database
travel_df = travel_df.rename(columns = {'Date':'date', 
                                        'Number of Trips 1-3 (Sum)':'num_trips_1_to_3', 
                                        'Number of Trips 3-5 (Sum)':'num_trips_3_to_5',
                                        'Number of Trips 5-10 (Sum)':'num_trips_5_to_10',
                                        'Number of Trips 10-25 (Sum)':'num_trips_10_to_25',
                                        'Number of Trips 25-50 (Sum)':'num_trips_25_to_50',
                                        'Number of Trips 50-100 (Sum)':'num_trips_50_to_100'})
travel_df.head()

,date,num_trips_1_to_3,num_trips_3_to_5,num_trips_5_to_10,num_trips_10_to_25,num_trips_25_to_50,num_trips_50_to_100
0,Jan-19,"26,834,466,455","12,952,315,594","15,950,096,166","15,199,029,960","5,076,183,145","1,676,952,407"
1,Feb-19,"24,347,566,662","11,870,026,967","14,719,257,863","14,093,668,108","4,715,357,077","1,572,076,828"
2,Mar-19,"30,942,105,172","14,991,418,422","18,737,410,376","18,267,432,391","5,967,630,335","1,902,728,755"
3,Apr-19,"33,425,468,112","16,067,286,787","20,196,735,426","20,041,766,804","6,405,800,428","1,924,130,243"
4,May-19,"34,318,175,900","16,488,446,849","20,778,297,665","20,303,221,871","6,116,174,341","1,740,648,434"


In [6]:
# read second csv
wfh_wfo_csv = "Resources/WFM_vs_WFO.csv"
wfh_wfo_df = pd.read_csv(wfh_wfo_csv)
wfh_wfo_df.head(30)

,Date,Population Staying at Home (Sum),Population Not Staying at Home (Sum)
0,Jan-19,"5,859,975,484","24,537,519,163"
1,Feb-19,"5,207,503,173","22,248,342,237"
2,Mar-19,"5,872,201,489","24,525,157,734"
3,Apr-19,"5,774,928,693","23,641,311,715"
4,May-19,"5,775,248,495","24,622,036,668"
5,Jun-19,"5,671,275,146","23,745,369,996"
6,Jul-19,"5,908,602,576","24,487,847,624"
7,Aug-19,"5,591,569,586","24,805,389,091"
8,Sep-19,"5,677,692,486","23,737,905,385"
9,Oct-19,"6,022,569,361","24,366,016,748"


In [7]:
# rename columns for export to database
wfh_wfo_df = wfh_wfo_df.rename(columns = {"Date": "date", 
                                          "Population Staying at Home (Sum)":"population_at_home", 
                                          "Population Not Staying at Home (Sum)":"population_not_at_home"})
wfh_wfo_df.head()

,date,population_at_home,population_not_at_home
0,Jan-19,"5,859,975,484","24,537,519,163"
1,Feb-19,"5,207,503,173","22,248,342,237"
2,Mar-19,"5,872,201,489","24,525,157,734"
3,Apr-19,"5,774,928,693","23,641,311,715"
4,May-19,"5,775,248,495","24,622,036,668"


In [8]:
# read third csv
csv_file = "Resources/WFH_WFO_dataset.csv"
wfhwfo_df = pd.read_csv(csv_file)
wfhwfo_df.head()

,ID,Name,Age,Occupation,Gender,Same_ofiice_home_location,kids,RM_save_money,RM_quality_time,RM_better_sleep,calmer_stressed,RM_professional_growth,RM_lazy,RM_productive,digital_connect_sufficient,RM_better_work_life_balance,RM_improved_skillset,RM_job_opportunities,Target
0,1,Bhavana,45,Tutor,Female,Yes,Yes,Yes,Yes,Yes,CALMER,5,1,5,Yes,5,5,Yes,1
1,2,Harry,24,Tutor,Male,No,No,No,No,No,CALMER,2,2,2,No,3,3,No,1
2,3,Banditaa,53,HR,Female,Yes,Yes,Yes,Yes,Yes,CALMER,3,3,4,No,5,3,No,1
3,4,Neetha,26,Engineer,Female,Yes,No,Yes,Yes,No,STRESSED,3,4,4,No,4,5,Yes,0
4,5,Ram,26,Recruiter,Male,Yes,No,No,Yes,No,STRESSED,3,3,5,Yes,4,2,Yes,1


In [9]:
# find columns
wfhwfo_df.columns

Index(['ID', 'Name', 'Age', 'Occupation', 'Gender',
       'Same_ofiice_home_location', 'kids', 'RM_save_money', 'RM_quality_time',
       'RM_better_sleep', 'calmer_stressed', 'RM_professional_growth',
       'RM_lazy', 'RM_productive', 'digital_connect_sufficient',
       'RM_better_work_life_balance', 'RM_improved_skillset',
       'RM_job_opportunities', 'Target'],
      dtype='object')

In [11]:
# rename columns for export to database
wfhwfo_df = wfhwfo_df.rename(columns = {'ID': 'id',
 'Name': 'name',
 'Age': 'age',
 'Occupation': 'occupation',
 'Gender': 'gender',
 'Same_ofiice_home_location': 'same_office_home_location',
 'kids': 'kids',
 'RM_save_money': 'rm_save_money',
 'RM_quality_time': 'rm_quality_time',
 'RM_better_sleep': 'rm_better_sleep',
 'calmer_stressed': 'calmer_stressed',
 'RM_professional_growth': 'rm_professional_growth',
 'RM_lazy': 'rm_lazy',
 'RM_productive': 'rm_productive',
 'digital_connect_sufficient': 'digital_connect_sufficient',
 'RM_better_work_life_balance': 'rm_better_work_life_balance',
 'RM_improved_skillset': 'rm_improved_skillset',
 'RM_job_opportunities': 'rm_job_opportunities',
 'Target': 'target'})


In [13]:
# drop unneeded columns
wfhwfo_df = wfhwfo_df.drop(['name','calmer_stressed','digital_connect_sufficient'], axis= 1)
wfhwfo_df.head()

,id,age,occupation,gender,same_office_home_location,kids,rm_save_money,rm_quality_time,rm_better_sleep,rm_professional_growth,rm_lazy,rm_productive,rm_better_work_life_balance,rm_improved_skillset,rm_job_opportunities,target
0,1,45,Tutor,Female,Yes,Yes,Yes,Yes,Yes,5,1,5,5,5,Yes,1
1,2,24,Tutor,Male,No,No,No,No,No,2,2,2,3,3,No,1
2,3,53,HR,Female,Yes,Yes,Yes,Yes,Yes,3,3,4,5,3,No,1
3,4,26,Engineer,Female,Yes,No,Yes,Yes,No,3,4,4,4,5,Yes,0
4,5,26,Recruiter,Male,Yes,No,No,Yes,No,3,3,5,4,2,Yes,1


In [14]:
# read fourth csv
csv_file = "Resources/2020_rws.csv"
data2020_df = pd.read_csv(csv_file)
data2020_df.head()

,Response ID,What year were you born?,What is your gender?,Which of the following best describes your industry?,Which of the following best describes your industry? (Detailed),Which of the following best describes your current occupation?,Which of the following best describes your current occupation? (Detailed),How many people are currently employed by your organisation?,Do you manage people as part of your current occupation?,Which of the following best describes your household?,...,Compare remote working to working at your employer’s workplace. Select the best aspect of remote working for you - Managing my family responsibilities ; My working relationships ; Preparing for work and commuting ; Managing my personal commitments ; My opportunities to socialise ; My mental wellbeing,Compare remote working to working at your employer’s workplace. Select the worst aspect of remote working for you - Managing my family responsibilities ; My working relationships ; Preparing for work and commuting ; Managing my personal commitments ; My opportunities to socialise ; My mental wellbeing,Compare remote working to working at your employer’s workplace. Select the best aspect of remote working for you - Managing my family responsibilities ; My working relationships ; Preparing for work and commuting ; My daily expenses ; My personal relationships ; My job satisfaction,Compare remote working to working at your employer’s workplace. Select the worst aspect of remote working for you - Managing my family responsibilities ; My working relationships ; Preparing for work and commuting ; My daily expenses ; My personal relationships ; My job satisfaction,Compare remote working to working at your employer’s workplace. Select the best aspect of remote working for you - The number of hours I work ; My work-life balance ; My on-the-job learning opportunities ; Managing my personal commitments ; My opportunities to socialise ; My mental wellbeing,Compare remote working to working at your employer’s workplace. Select the worst aspect of remote working for you - The number of hours I work ; My work-life balance ; My on-the-job learning opportunities ; Managing my personal commitments ; My opportunities to socialise ; My mental wellbeing,Compare remote working to working at your employer’s workplace. Select the best aspect of remote working for you - The number of hours I work ; My work-life balance ; My on-the-job learning opportunities ; My daily expenses ; My personal relationships ; My job satisfaction,Compare remote working to working at your employer’s workplace. Select the worst aspect of remote working for you - The number of hours I work ; My work-life balance ; My on-the-job learning opportunities ; My daily expenses ; My personal relationships ; My job satisfaction,Compare remote working to working at your employer’s workplace. Select the best aspect of remote working for you - Managing my personal commitments ; My opportunities to socialise ; My mental wellbeing ; My daily expenses ; My personal relationships ; My job satisfaction,Compare remote working to working at your employer’s workplace. Select the worst aspect of remote working for you - Managing my personal commitments ; My opportunities to socialise ; My mental wellbeing ; My daily expenses ; My personal relationships ; My job satisfaction
0,1,1972,Female,Manufacturing,Food Product Manufacturing,Clerical and administrative,Other Clerical and Administrative,Between 20 and 199,No,Couple with no dependent children,...,Managing my personal commitments,My opportunities to socialise,Preparing for work and commuting,My working relationships,Managing my personal commitments,The number of hours I work,My job satisfaction,The number of hours I work,Managing my personal commitments,My opportunities to socialise
1,2,1972,Male,Wholesale Trade,Other Goods Wholesaling,Managers,"Chief Executives, General Managers and Legisla...",Between 1 and 4,Yes,Couple with dependent children,...,Preparing for work

In [15]:
# explore csv columns
data2020_df.columns

Index(['Response ID', 'What year were you born?', 'What is your gender?',
       'Which of the following best describes your industry?',
       'Which of the following best describes your industry? (Detailed)',
       'Which of the following best describes your current occupation?',
       'Which of the following best describes your current occupation? (Detailed)',
       'How many people are currently employed by your organisation?',
       'Do you manage people as part of your current occupation?',
       'Which of the following best describes your household?',
       'How long have you been in your current job?', 'Metro / Regional',
       'Thinking about your current job, how much of your time did you spend remote working last year?',
       'Thinking about remote working last year, how strongly do you agree or disagree with the following statements? - My organisation encouraged people to work remotely',
       'Thinking about remote working last year, how strongly do you agree or 

In [16]:
# rename to shorten column names
newdata2020_df = data2020_df.rename(columns={'Response ID':'ID','What year were you born?':'Year_born',
                                              'What is your gender?':'Gender',
                                              'Which of the following best describes your industry?':'Industry',
                                               'Which of the following best describes your current occupation? (Detailed)':'Occupation',
                                               'How many people are currently employed by your organisation?':'Num_of_employees',
                                                'Do you manage people as part of your current occupation?':'Manager',
                                                 'Which of the following best describes your household?':'Household_type',
                                             'How long have you been in your current job?': 'Experience','Metro / Regional':'Proximity',
                                             'Thinking about your current job, how much of your time did you spend remote working last year?':'Remote_work_last_year',
       'Thinking about your current job, how much of your time did you spend remote working in the last 3 months?':'Remote_work_last_quarter',
                                             'How much of your time would you have preferred to work remotely last year?':'Last_year_remote_preference',
                                             'How much of your time would you have preferred to work remotely in the last 3 months?':'Last_quarter_remote_preference',
       'Imagine that COVID-19 is cured or eradicated. Going forward, how much of your time would you prefer to work remotely?':'Future_remote_preference',
                                             'Thinking about remote working in the last 3 months, how strongly do you agree or disagree with the following statements? - I would recommend remote working to others':'Recommend_remote',
                                             'This question is about your productivity. Productivity means what you produce for each hour that you work. It includes the amount of work you achieve each hour, and the quality of your work each hour.  \nPlease compare your productivity when you work remotely to when you work at your employer’s workplace.  \nRoughly how productive are you, each hour, when you work remotely?':'Productivity',
                                               'Compare remote working to working at your employer’s workplace. Select the best aspect of remote working for you - Managing my family responsibilities ; My working relationships ; Preparing for work and commuting ; Managing my personal commitments ; My opportunities to socialise ; My mental wellbeing':'bestaspect1',
                                              'Compare remote working to working at your employer’s workplace. Select the worst aspect of remote working for you - Managing my family responsibilities ; My working relationships ; Preparing for work and commuting ; Managing my personal commitments ; My opportunities to socialise ; My mental wellbeing':'worstaspect1',
                                             'Compare remote working to working at your employer’s workplace. Select the best aspect of remote working for you - Managing my family responsibilities ; My working relationships ; Preparing for work and commuting ; My daily expenses ; My personal relationships ; My job satisfaction':'bestaspect2',
                                             'Compare remote working to working at your employer’s workplace. Select the worst aspect of remote working for you - Managing my family responsibilities ; My working relationships ; Preparing for work and commuting ; My daily expenses ; My personal relationships ; My job satisfaction':'worstaspect2'})

In [17]:
# keep applicable columns
cleandata2020_df = newdata2020_df[['ID',
                                   'Year_born',
                                   'Gender',
                                   'Industry',
                                   'Occupation',
                                   'Experience',
                                   'Num_of_employees',
                                   'Manager',
                                   'Proximity',
                                   'Household_type',
                                   'Remote_work_last_year',
                                   'Remote_work_last_quarter',
                                   'Recommend_remote',
                                   'Last_year_remote_preference',
                                   'Last_quarter_remote_preference',
                                   'Future_remote_preference',
                                   'Productivity',
                                   'bestaspect1',
                                   'worstaspect1',
                                   'bestaspect2',
                                   'worstaspect2']]
cleandata2020_df.head()

,ID,Year_born,Gender,Industry,Occupation,Experience,Num_of_employees,Manager,Proximity,Household_type,...,Remote_work_last_quarter,Recommend_remote,Last_year_remote_preference,Last_quarter_remote_preference,Future_remote_preference,Productivity,bestaspect1,worstaspect1,bestaspect2,worstaspect2
0,1,1972,Female,Manufacturing,Other Clerical and Administrative,More than 5 years,Between 20 and 199,No,Regional,Couple with no dependent children,...,90%,Strongly agree,I would not have preferred to work remotely,80%,90%,I’m 50% more productive when working remotely ...,Managing my personal commitments,My opportunities to socialise,Preparing for work and commuting,My working relationships
1,2,1972,Male,Wholesale Trade,"Chief Executives, General Managers and Legisla...",More than 5 years,Between 1 and 4,Yes,Metro,Couple with dependent children,...,20%,Somewhat agree,20%,20%,20%,I’m 20% less productive when working remotely,Preparing for work and commuting,My working relationships,Preparing for work and commuting,My working relationships
2,3,1982,Male,"Electricity, Gas, Water and Waste Services","Chief Executives, General Managers and Legisla...",More than 5 years,More than 200,Yes,Metro,One parent family with dependent children,...,50% - I spent about half of my time remote wor...,Neither agree nor disagree,50% - About half of my time,60%,60%,I’m 10% less productive when working remotely,Managing my personal commitments,Preparing for work and commuting,Preparing for work and commuting,Managing my family responsibilities
3,4,1987,Female,"Professional, Scientific and Technical Services",ICT Professionals,Between 1 and 5 years,Between 20 and 199,No,Metro,Couple with dependent children,...,100% - I spent all of my time remote working,Strongly agree,50% - About half of my time,100% - All of my time,100% - All of my time,My productivity is about same when I work remo...,Preparing for work and commuting,My opportunities to socialise,My personal relationships,My working relationships
4,5,1991,Male,"Transport, Postal and Warehousing",Specialist Managers,More than 5 years,Between 5 and 19,Yes,Metro,Couple with no dependent children,...,90%,Strongly agree,60%,100% - All of my time,60%,I’m 20% more productive when working remotely,Managing my personal commitments,My working relationships,Preparing for work and commuting,My daily expenses


In [19]:
# rename columns for export to database
cleandata2020_df = cleandata2020_df.rename(columns ={'ID': 'id',
 'Year_born': 'year_born',
 'Gender': 'gender',
 'Industry': 'industry',
 'Occupation': 'occupation',
 'Experience': 'experience',
 'Num_of_employees': 'num_of_employees',
 'Manager': 'manager',
 'Proximity': 'proximity',
 'Household_type': 'household_type',
 'Remote_work_last_year': 'remote_work_last_year',
 'Remote_work_last_quarter': 'remote_work_last_quarter',
 'Recommend_remote': 'recommend_remote',
 'Last_year_remote_preference': 'last_year_remote_preference',
 'Last_quarter_remote_preference': 'last_quarter_remote_preference',
 'Future_remote_preference': 'future_remote_preference',
 'Productivity': 'productivity',
 'bestaspect1': 'bestaspect1',
 'worstaspect1': 'worstaspect1',
 'bestaspect2': 'bestaspect2',
 'worstaspect2': 'worstaspect2'} )

In [20]:
# read fifth csv
csv_file21 = "Resources/2021_rws.csv"
data2021_df = pd.read_csv(csv_file21)
data2021_df.head()

,Response ID,What year were you born?,What is your gender?,"How long have you been in your current job? If you have more than one job, please answer all questions for your main job.",Which of the following best describes your industry?,Which of the following best describes your current occupation?,How many people are currently employed by your organisation?,Which of the following best describes your household?,Metro or Regional,"Thinking about your current job, how much of your work time did you spend working remotely in the last quarter of last year? This means October-December 2020 If you work a 5 day week, each day of remote working equals 20% of your time.",...,How strongly do you agree or disagree with the following statements? - Offering remote work helps me retain employees,How strongly do you agree or disagree with the following statements? - Offering remote work helps me to recruit employees,How strongly do you agree or disagree with the following statements? - My team works well together when they work remotely,How strongly do you agree or disagree with the following statements? - I find it easy to manage employees remotely (e.g. tasking and monitoring progress),How strongly do you agree or disagree with the following statements? - I find it easy to manage poor performers remotely,How strongly do you agree or disagree with the following statements? - I feel well-prepared to manage employees remotely,How strongly do you agree or disagree with the following statements? - Managing people remotely makes me more focused on results,How strongly do you agree or disagree with the following statements? - I find it easy to contact my employees when they work remotely,"This question is about your productivity. Productivity means what you produce for each hour that you work. It includes the amount of work you achieve each hour, and the quality of your work each hour. Please compare your productivity when you work remotely to when you work at your employer’s workplace. Roughly how productive are you, each hour, when you work remotely?","Now think about the productivity of the employees you manage. Roughly how productive are the employees you manage, each hour, when they work remotely?"
0,1,1976,Male,More than 5 years,Other Services,Professionals - ICT Professionals,More than 200,Couple with dependent children,Metro,50% - I spent about half of my time remote wor...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I’m 50% more productive when working remotely ...,NaN
1,2,1971,Male,More than 5 years,Construction,Professionals - ICT Professionals,More than 200,Couple with no dependent children,Metro,10%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I’m 50% more productive when working remotely ...,NaN
2,3,1978,Female,Between 6 and 12 months,Financial and Insurance,"Professionals - Business, Human Resource and M...",More than 200,Single person,Metro,100% - I spent all of my time remote working,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,My productivity is about same when I work remo...,NaN
3,4,1962,Female,Between 1 and 5 years,Health Care and Social Assistance,Professionals - Health Professionals,Between 20 and 199,One parent family with dependent children,Regional,40%,...,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,I’m 50% more productive when working remotely ...,I’m 50% more productive when working remotely ...
4,5,1974,Male,More than 5 years,Financial and Insurance,Managers - Specialist Managers,More than 200,Couple with dependent children,Regional,100% - I spent all of my time remote working,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I’m 20% more productive when working remotely,NaN


In [21]:
# explore columns
data2021_df.columns

Index(['Response ID', 'What year were you born?  ', 'What is your gender?  ',
       'How long have you been in your current job?  If you have more than one job, please answer all questions for your main job.  ',
       'Which of the following best describes your industry?  ',
       'Which of the following best describes your current occupation?  ',
       'How many people are currently employed by your organisation?   ',
       'Which of the following best describes your household?  ',
       'Metro or Regional',
       'Thinking about your current job, how much of your work time did you spend working remotely in the last quarter of last year?   This means October-December 2020  If you work a 5 day week, each day of remote working equals 20% of your time.  ',
       ...
       'How strongly do you agree or disagree with the following statements?    - Offering remote work helps me retain employees',
       'How strongly do you agree or disagree with the following statements?    - Offe

In [22]:
# rename to shorten column names
newdata2021_df = data2021_df.rename(columns={'Response ID':'ID', 'What year were you born?  ':'Year_born', 'What is your gender?  ':'Gender',
       'How long have you been in your current job?  If you have more than one job, please answer all questions for your main job.  ':'Experience',
       'Which of the following best describes your industry?  ':'Industry',
       'Which of the following best describes your current occupation?  ':'Occupation',
       'How many people are currently employed by your organisation?   ':'Num_of_employees',
       'Which of the following best describes your household?  ':'Household_type',
       'Metro or Regional':'Proximity','Thinking about your current job, how much of your work time did you spend working remotely in the last quarter of last year?   This means October-December 2020  If you work a 5 day week, each day of remote working equals 20% of your time.  ':'Remote_work_last_quarter',
                                             'How much of your work time would you have preferred to work remotely during the last quarter of last year?   This means October-December 2020  If you work a 5 day week, each day of remote working equals 20% of your time.  ':'Last_quarter_remote_preference',
                                             'Thinking about your current job, how much of your work time have you spent working remotely this year?  If you work a 5 day week, each day of remote working equals 20% of your time.  ':'Remote_work_last_year',
                                             'How much of your work time would you have preferred to work remotely so far this year?  If you work a 5 day week, each day of remote working equals 20% of your time.  ':'Last_year_remote_preference',
                                             'Imagine that COVID-19 is cured or eradicated.   Going forward, how much of your work time would you prefer to work remotely?  If you work a 5 day week, each day of remote working equals 20% of your time.  ':'Future_remote_preference',
       'Thinking about your current job, how much of your work time did you spend working remotely in the last quarter of last year?   This means October-December 2020  If you work a 5 day week, each day of remote working equals 20% of your time.  ':'Remote_work_last_quarter',
                                             'This question is about your productivity. Productivity means what you produce for each hour that you work. It includes the amount of work you achieve each hour, and the quality of your work each hour.    Please compare your productivity when you work remotely to when you work at your employer’s workplace.    Roughly how productive are you, each hour, when you work remotely?  ':'Productivity'})

In [23]:
# keep columns appropriate to project
cleandata2021_df = newdata2021_df[['ID','Year_born','Gender','Industry','Occupation','Experience','Num_of_employees','Household_type','Proximity','Remote_work_last_year','Remote_work_last_quarter', 'Last_year_remote_preference', 'Last_quarter_remote_preference','Future_remote_preference','Productivity',
                                 ]]
cleandata2021_df.head()

,ID,Year_born,Gender,Industry,Occupation,Experience,Num_of_employees,Household_type,Proximity,Remote_work_last_year,Remote_work_last_quarter,Last_year_remote_preference,Last_quarter_remote_preference,Future_remote_preference,Productivity
0,1,1976,Male,Other Services,Professionals - ICT Professionals,More than 5 years,More than 200,Couple with dependent children,Metro,50% - I spent about half of my time remote wor...,50% - I spent about half of my time remote wor...,80%,80%,80%,I’m 50% more productive when working remotely ...
1,2,1971,Male,Construction,Professionals - ICT Professionals,More than 5 years,More than 200,Couple with no dependent children,Metro,Less than 10% of my time,10%,50% - About half of my time,50% - About half of my time,50% - About half of my time,I’m 50% more productive when working remotely ...
2,3,1978,Female,Financial and Insurance,"Professionals - Business, Human Resource and M...",Between 6 and 12 months,More than 200,Single person,Metro,90%,100% - I spent all of my time remote working,90%,100% - All of my time,60%,My productivity is about same when I work remo...
3,4,1962,Female,Health Care and Social Assistance,Professionals - Health Professionals,Between 1 and 5 years,Between 20 and 199,One parent family with dependent children,Regional,40%,40%,40%,40%,40%,I’m 50% more productive when working remotely ...
4,5,1974,Male,Financial and Insurance,Managers - Specialist Managers,More than 5 years,More than 200,Couple with dependent children,Regional,100% - I spent all of my time remote working,100% - I spent all of my time remote working,100% - All of my time,100% - All of my time,100% - All of my time,I’m 20% more productive when working remotely


In [25]:
# rename columns for export to database
cleandata2021_df = cleandata2021_df.rename(columns = {'ID': 'id',
 'Year_born': 'year_born',
 'Gender': 'gender',
 'Industry': 'industry',
 'Occupation': 'occupation',
 'Experience': 'experience',
 'Num_of_employees': 'num_of_employees',
 'Household_type': 'household_type',
 'Proximity': 'proximity',
 'Remote_work_last_year': 'remote_work_last_year',
 'Remote_work_last_quarter': 'remote_work_last_quarter',
 'Last_year_remote_preference': 'last_year_remote_preference',
 'Last_quarter_remote_preference': 'last_quarter_remote_preference',
 'Future_remote_preference': 'future_remote_preference',
 'Productivity': 'productivity'})

In [26]:
# Keep only columns in common between 2020 and 2021 datasets
rws_2020 = cleandata2020_df[[
                             'year_born',
                             'gender',
                             'industry',
                             'occupation',
                             'experience',
                             'num_of_employees',
                             'household_type',
                             'proximity',
                             'remote_work_last_year',
                             'remote_work_last_quarter',
                             'last_year_remote_preference',
                             'last_quarter_remote_preference',
                             'future_remote_preference',
                             'productivity',
                                  ]]

In [27]:
# Keep only columns in common between 2020 and 2021 datasets
rws_2021 = cleandata2021_df[[
                             'year_born',
                             'gender',
                             'industry',
                             'occupation',
                             'experience',
                             'num_of_employees',
                             'household_type',
                             'proximity',
                             'remote_work_last_year',
                             'remote_work_last_quarter',
                             'last_year_remote_preference',
                             'last_quarter_remote_preference',
                             'future_remote_preference',
                             'productivity',
                                ]]

In [28]:
# combine 2020 and 2021 datasets into one table
remote_work_comb_df = pd.concat([rws_2020,rws_2021])
remote_work_comb_df

,year_born,gender,industry,occupation,experience,num_of_employees,household_type,proximity,remote_work_last_year,remote_work_last_quarter,last_year_remote_preference,last_quarter_remote_preference,future_remote_preference,productivity
0,1972,Female,Manufacturing,Other Clerical and Administrative,More than 5 years,Between 20 and 199,Couple with no dependent children,Regional,Rarely or never,90%,I would not have preferred to work remotely,80%,90%,I’m 50% more productive when working remotely ...
1,1972,Male,Wholesale Trade,"Chief Executives, General Managers and Legisla...",More than 5 years,Between 1 and 4,Couple with dependent children,Metro,20%,20%,20%,20%,20%,I’m 20% less productive when working remotely
2,1982,Male,"Electricity, Gas, Water and Waste Services","Chief Executives, General Managers and Legisla...",More than 5 years,More than 200,One parent family with dependent children,Metro,60%,50% - I spent about half of my time remote wor...,50% - About half of my time,60%,60%,I’m 10% less productive when working remotely
3,1987,Female,"Professional, Scientific and Technical Services",ICT Professionals,Between 1 and 5 years,Between 20 and 199,Couple with dependent children,Metro,20%,100% - I spent all of my time remote working,50% - About half of my time,100% - All of my time,100% - All of my time,My productivity is about same when I work remo...
4,1991,Male,"Transport, Postal and Warehousing",Specialist Managers,More than 5 years,Between 5 and 19,Couple with no dependent children,Metro,Less than 10% of my time,90%,60%,100% - All of my time,60%,I’m 20% more productive when working remotely
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,1995,Male,Wholesale Trade,"Managers - Chief Executives, General Managers ...",Between 1 and 5 years,Between 5 and 19,Couple with dependent children,Metro,30%,30%,30%,40%,30%,I’m 30% more productive when working remotely
1508,1993,Female,Health Care and Social Assistance,Professionals - Health Professionals,More than 5 years,Between 5 and 19,Multiple family household,Metro,Less than 10% of my time,Less than 10% of my time,I would not have preferred to work remotely,I would not have preferred to work remotely,I would prefer not to work remotely,My productivity is about same when I work remo...
1509,1979,Female,Administrative and Support,Clerical and administrative workers - Clerical...,More than 5 years,Between 20 and 199,One parent family with dependent children,Regional,Rarely or never,Rarely or never,90%,100% - All of my time,90%,My productivity is about same when I work remo...
1510,1965,Female,Education and Training,"Managers - Chief Executives, General Managers ...",More than 5 years,Between 1 and 4,Couple with no dependent children,Metro,Less than 10% of my time,Less than 10% of my time,Less than 10% of my time,Less than 10% of my time,Less than 10% of my time,My productivity is about same when I work remo...


In [29]:
# add a new column to calculate age from birth year
remote_work_comb_df['age']= (2022 - remote_work_comb_df['year_born'])

remote_work_comb_df.head()

,year_born,gender,industry,occupation,experience,num_of_employees,household_type,proximity,remote_work_last_year,remote_work_last_quarter,last_year_remote_preference,last_quarter_remote_preference,future_remote_preference,productivity,age
0,1972,Female,Manufacturing,Other Clerical and Administrative,More than 5 years,Between 20 and 199,Couple with no dependent children,Regional,Rarely or never,90%,I would not have preferred to work remotely,80%,90%,I’m 50% more productive when working remotely ...,50
1,1972,Male,Wholesale Trade,"Chief Executives, General Managers and Legisla...",More than 5 years,Between 1 and 4,Couple with dependent children,Metro,20%,20%,20%,20%,20%,I’m 20% less productive when working remotely,50
2,1982,Male,"Electricity, Gas, Water and Waste Services","Chief Executives, General Managers and Legisla...",More than 5 years,More than 200,One parent family with dependent children,Metro,60%,50% - I spent about half of my time remote wor...,50% - About half of my time,60%,60%,I’m 10% less productive when working remotely,40
3,1987,Female,"Professional, Scientific and Technical Services",ICT Professionals,Between 1 and 5 years,Between 20 and 199,Couple with dependent children,Metro,20%,100% - I spent all of my time remote working,50% - About half of my time,100% - All of my time,100% - All of my time,My productivity is about same when I work remo...,35
4,1991,Male,"Transport, Postal and Warehousing",Specialist Managers,More than 5 years,Between 5 and 19,Couple with no dependent children,Metro,Less than 10% of my time,90%,60%,100% - All of my time,60%,I’m 20% more productive when working remotely,31


In [30]:
# export to postgres
travel_df.to_sql(name='travel', con=engine, if_exists='replace', index=False)

In [31]:
# export to postgres
wfh_wfo_df.to_sql(name='wfh_vs_wfo', con=engine, if_exists='replace', index=False)

In [32]:
# export to postgres
wfhwfo_df.to_sql(name='wfh_preference', con=engine, if_exists='replace', index=False)

In [33]:
# export to postgres
cleandata2020_df.to_sql(name='rws_2020', con=engine, if_exists='replace', index=False)

In [34]:
# export to postgres
cleandata2021_df.to_sql(name='rws_2021', con=engine, if_exists='replace', index=False)

In [35]:
# export to postgres
remote_work_comb_df.to_sql(name='rws_comb', con=engine, if_exists='replace', index=False)

In [36]:
# check for successful creation of all tables
engine.table_names()

C:\Users\Hannah\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['wfh_vs_wfo', 'wfh_preference', 'travel', 'rws_2020', 'rws_2021', 'rws_comb']

In [37]:
# check for successful export of data to postgres
pd.read_sql_query('select * from travel', con=engine).head()

,date,num_trips_1_to_3,num_trips_3_to_5,num_trips_5_to_10,num_trips_10_to_25,num_trips_25_to_50,num_trips_50_to_100
0,Jan-19,"26,834,466,455","12,952,315,594","15,950,096,166","15,199,029,960","5,076,183,145","1,676,952,407"
1,Feb-19,"24,347,566,662","11,870,026,967","14,719,257,863","14,093,668,108","4,715,357,077","1,572,076,828"
2,Mar-19,"30,942,105,172","14,991,418,422","18,737,410,376","18,267,432,391","5,967,630,335","1,902,728,755"
3,Apr-19,"33,425,468,112","16,067,286,787","20,196,735,426","20,041,766,804","6,405,800,428","1,924,130,243"
4,May-19,"34,318,175,900","16,488,446,849","20,778,297,665","20,303,221,871","6,116,174,341","1,740,648,434"
